# Quality Assurance with RasCheck

This notebook demonstrates how to use the RasCheck module for validating HEC-RAS steady flow models.

In [ ]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
USE_LOCAL_SOURCE = False  # <-- TOGGLE THIS

if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)
    if local_path not in sys.path:
        sys.path.insert(0, local_path)
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

# Import ras-commander
from ras_commander import (
    RasExamples, init_ras_project, ras,
    RasCmdr, HdfResultsPlan
)
from ras_commander.check import (
    RasCheck, CheckResults, Severity,
    RasCheckReport, ReportMetadata,
    get_default_thresholds, create_custom_thresholds,
    get_state_surcharge_limit
)

# Additional imports
import pandas as pd
from pathlib import Path

# Set pandas display options
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Verify which version loaded
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

---

## Attribution

> **Based on FEMA cHECk-RAS**
>
> This module implements the validation checks described in FEMA's
> [cHECk-RAS](https://www.fema.gov/flood-maps/tutorials/check-ras) quality assurance tool
> for HEC-RAS 6.x models, using ras-commander's HDF function library.
>
> From FEMA's cHECk-RAS page:
>
> *"cHECk-RAS utilizes information generated by HEC-RAS (all versions through the latest
> version, 5.0.7). Note, cHECk-RAS is not compatible with the two-dimensional component
> of HEC-RAS 5.0.7."*
>
> This Python implementation extends these validation checks to work with modern HEC-RAS 6.x
> HDF-based outputs, providing equivalent functionality through the ras-commander library.

> **For Informational Purposes Only**
>
> Outputs from this library are for **informational purposes only** and should be
> validated by a qualified engineer. This tool does not replace professional engineering
> judgment or official FEMA review processes.

**NOTE**: This is an UNOFFICIAL Python implementation and is NOT affiliated with or endorsed by FEMA.

## Overview

The **RasCheck** module provides comprehensive quality assurance validation for HEC-RAS steady flow models. It implements the validation checks described in FEMA's cHECk-RAS using modern HDF-based data access for HEC-RAS 6.x models.

### Check Categories

| Check | Description | Key Validations |
|-------|-------------|------------------|
| **NT Check** | Manning's n and Transition | Roughness coefficients, contraction/expansion losses |
| **XS Check** | Cross Section Validation | Spacing, ineffective flow, reach lengths, levees |
| **Structure Check** | Bridges, Culverts, Weirs | Flow classification, coefficients, geometry |
| **Floodway Check** | Floodway Analysis | Surcharge limits, encroachment methods |
| **Profiles Check** | Multiple Profile Consistency | WSE ordering, discharge consistency |

In [4]:
# Initialize the project
init_ras_project(project_path, "6.6")
print(f"Initialized project: {ras.project_name}")

2025-12-13 20:15:33 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.rasmap


Initialized project: BaldEagle


In [5]:
# View available plans
print("Available plans:")
ras.plan_df[['plan_number', 'Plan Title', 'flow_type']]

Available plans:


,plan_number,Plan Title,flow_type
0,01,Unsteady with Bridges and Dam,Unsteady
1,02,Steady Flow Run,Steady


## Run the Steady Flow Plan

RasCheck requires computed results in the HDF file. We'll run the steady flow plan (Plan 02) if results don't exist.

In [6]:
# Use the steady flow plan
plan_number = "02"

# Execute the plan (skip if results already exist)
print(f"Running Plan {plan_number}...")
success = RasCmdr.compute_plan(plan_number, skip_existing=True)
if success:
    print(f"Plan {plan_number} ready for validation")
else:
    print(f"Plan {plan_number} execution failed - check HEC-RAS installation")

2025-12-13 20:15:33 - ras_commander.RasCmdr - INFO - Using ras_object with project folder: C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek
2025-12-13 20:15:33 - ras_commander.RasCmdr - INFO - Running HEC-RAS from the Command Line:
2025-12-13 20:15:33 - ras_commander.RasCmdr - INFO - Running command: "C:\Program Files (x86)\HEC\HEC-RAS\6.6\Ras.exe" -c "C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.prj" "C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.p02"


Running Plan 02...


2025-12-13 20:15:38 - ras_commander.RasCmdr - INFO - HEC-RAS execution completed for plan: 02
2025-12-13 20:15:38 - ras_commander.RasCmdr - INFO - Total run time for plan 02: 5.08 seconds


Plan 02 ready for validation


In [7]:
# Verify this is a steady state plan
is_steady = HdfResultsPlan.is_steady_plan(plan_number)
print(f"Is Plan {plan_number} a steady state plan? {is_steady}")

# Get profile names
profiles = HdfResultsPlan.get_steady_profile_names(plan_number)
print(f"\nFound {len(profiles)} profiles: {profiles}")

2025-12-13 20:15:38 - ras_commander.hdf.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.p02.hdf
2025-12-13 20:15:38 - ras_commander.hdf.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.p02.hdf
2025-12-13 20:15:38 - ras_commander.hdf.HdfResultsPlan - INFO - Found 8 steady state profiles: ['.5 year', '1 year', '2 year', '5 year', '10 year', '25 year', '50 year', '100 year']


Is Plan 02 a steady state plan? True

Found 8 profiles: ['.5 year', '1 year', '2 year', '5 year', '10 year', '25 year', '50 year', '100 year']


---

## Running RasCheck Validation

### Option 1: Run All Checks at Once

The simplest approach is to run all checks in one call:

In [8]:
# Run all validation checks
results = RasCheck.run_all(plan_number)

# View summary
print(f"Validation Results:")
print(f"  Total Messages: {len(results.messages)}")
print(f"  Errors: {results.get_error_count()}")
print(f"  Warnings: {results.get_warning_count()}")

2025-12-13 20:15:38 - ras_commander.hdf.HdfXsec - ERROR - Error processing cross-section data: [Errno 2] Unable to synchronously open file (unable to open file: name = 'C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.hdf', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
2025-12-13 20:15:38 - ras_commander.hdf.HdfXsec - ERROR - Error processing cross-section data: [Errno 2] Unable to synchronously open file (unable to open file: name = 'C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.hdf', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
2025-12-13 20:15:38 - ras_commander.hdf.HdfStruc - ERROR - HDF file not found: C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.hdf
2025-12-13 20:15:38 - ras_commander.check.RasCheck - WARNING - Could not read structures: HDF f

Validation Results:
  Total Messages: 4145
  Errors: 1
  Warnings: 3671


### Option 2: Run Individual Checks

You can also run specific checks independently for more control:

In [9]:
# Get HDF paths for individual checks
plan_row = ras.plan_df[ras.plan_df['plan_number'] == plan_number].iloc[0]
plan_hdf = Path(plan_row['HDF_Results_Path'])
geom_hdf = Path(plan_row['Geom Path']).with_suffix('.hdf')

print(f"Plan HDF: {plan_hdf}")
print(f"Geom HDF: {geom_hdf}")

Plan HDF: C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.p02.hdf
Geom HDF: C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.hdf


In [10]:
# Run NT Check (Manning's n and Transition coefficients)
nt_results = RasCheck.check_nt(geom_hdf)
print(f"NT Check: {len(nt_results.messages)} messages")
print(f"  Errors: {nt_results.get_error_count()}")
print(f"  Warnings: {nt_results.get_warning_count()}")

2025-12-13 20:15:38 - ras_commander.hdf.HdfXsec - ERROR - Error processing cross-section data: [Errno 2] Unable to synchronously open file (unable to open file: name = 'C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.hdf', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


NT Check: 1 messages
  Errors: 1
  Warnings: 0


---

## Analyzing Results

### Convert Results to DataFrame

In [11]:
# Convert all messages to DataFrame for analysis
df = results.to_dataframe()

if not df.empty:
    print(f"Results DataFrame shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    display(df.head(10))
else:
    print("No validation messages generated")

Results DataFrame shape: (4145, 12)

Columns: ['message_id', 'severity', 'check_type', 'river', 'reach', 'station', 'structure', 'message', 'flagged', 'comment', 'value', 'threshold']


,message_id,severity,check_type,river,reach,station,structure,message,flagged,comment,value,threshold
0,SYS_003,ERROR,SYSTEM,,,,,No cross section data found in geometry HDF,False,,NaN,None
1,MP_WS_01,WARNING,PROFILES,Bald Eagle,Loc Hav,138154.4,,WSE for .5 year is less than 1 year at RS 1381...,False,,-0.843140,None
2,MP_Q_01,WARNING,PROFILES,Bald Eagle,Loc Hav,138154.4,,Discharge for .5 year is less than 1 year at R...,False,,-1000.000000,None
3,PF_VEL_01,WARNING,PROFILES,Bald Eagle,Loc Hav,138154.4,,Velocity for .5 year (3.66 ft/s) is less than ...,False,,-0.231416,None
4,MP_WS_01,WARNING,PROFILES,Bald Eagle,Loc Hav,138154.4,,WSE for 1 year is less than 2 year at RS 138154.4,False,,-1.170532,None
5,MP_Q_01,WARNING,PROFILES,Bald Eagle,Loc Hav,138154.4,,Discharge for 1 year is less than 2 year at RS...,False,,-2000.000000,None
6,PF_VEL_01,WARNING,PROFILES,Bald Eagle,Loc Hav,138154.4,,Velocity for 1 year (3.89 ft/s) is less than 2...,False,,-0.631384,None
7,MP_WS_01,WARNING,PROFILES,Bald Eagle,Loc Hav,138154.4,,WSE for 2 year is less than 5 year at RS 138154.4,False,,-2.070618,None
8,MP_Q_01,WARNING,PROFILES,Bald Eagle,Loc Hav,138154.4,,Discharge for 2 year is less than 5 year at RS...,False,,-5000.000000,None
9,PF_VEL_01,WARNING,PROFILES,Bald Eagle,Loc Hav,138154.4,,Velocity for 2 year (4.52 ft/s) is less than 5...,False,,-1.271657,None


### Filter Messages by Severity

In [12]:
# Get only ERROR messages
errors = results.filter_by_severity(Severity.ERROR)
print(f"Found {len(errors)} ERROR messages:")

for msg in errors[:5]:  # Show first 5
    print(f"  [{msg.message_id}] RS {msg.station}: {msg.message}")

Found 1 ERROR messages:
  [SYS_003] RS : No cross section data found in geometry HDF


In [13]:
# Get only WARNING messages
warnings = results.filter_by_severity(Severity.WARNING)
print(f"Found {len(warnings)} WARNING messages:")

for msg in warnings[:5]:  # Show first 5
    print(f"  [{msg.message_id}] RS {msg.station}: {msg.message}")

Found 3671 WARNING messages:
  [MP_WS_01] RS 138154.4: WSE for .5 year is less than 1 year at RS 138154.4
  [MP_Q_01] RS 138154.4: Discharge for .5 year is less than 1 year at RS 138154.4
  [PF_VEL_01] RS 138154.4: Velocity for .5 year (3.66 ft/s) is less than 1 year (3.89 ft/s) at RS 138154.4
  [MP_WS_01] RS 138154.4: WSE for 1 year is less than 2 year at RS 138154.4
  [MP_Q_01] RS 138154.4: Discharge for 1 year is less than 2 year at RS 138154.4


### Filter Messages by Check Type

In [14]:
# Group messages by check type
if not df.empty:
    check_summary = df.groupby(['check_type', 'severity']).size().unstack(fill_value=0)
    print("Messages by Check Type and Severity:")
    display(check_summary)

Messages by Check Type and Severity:


severity,ERROR,INFO,WARNING
check_type,,,
PROFILES,0,473,3671
SYSTEM,1,0,0


In [15]:
# Filter by specific check type
nt_msgs = results.filter_by_check_type("NT")
print(f"\nNT Check Messages ({len(nt_msgs)} total):")

# Show message ID distribution
if nt_msgs:
    msg_ids = [m.message_id for m in nt_msgs]
    print(pd.Series(msg_ids).value_counts())


NT Check Messages (0 total):


### Filter Messages by Station

In [16]:
# Get unique stations with issues
if not df.empty:
    stations_with_issues = df['station'].unique()
    print(f"Stations with issues: {len(stations_with_issues)}")
    print(f"\nFirst 10 stations: {list(stations_with_issues[:10])}")

Stations with issues: 179

First 10 stations: ['', '138154.4', '137690.8', '137327.0', '136564.9', '136202.3', '135591.4', '135068.7', '134487.2', '133881.0']


In [17]:
# Filter messages for a specific station
if not df.empty and len(stations_with_issues) > 0:
    sample_station = stations_with_issues[0]
    station_msgs = results.filter_by_station(sample_station)
    
    print(f"\nMessages for station {sample_station}:")
    for msg in station_msgs:
        print(f"  [{msg.severity.value}] {msg.message_id}: {msg.message}")


Messages for station :
  [ERROR] SYS_003: No cross section data found in geometry HDF


---

## Generating Reports

### HTML Report

Generate a comprehensive HTML report with all validation results:

In [18]:
# Define output path for report
report_path = project_path / "ras_checker" / "validation_report.html"
report_path.parent.mkdir(parents=True, exist_ok=True)

# Generate HTML report
output_path = results.to_html(report_path)
print(f"HTML report generated: {output_path}")

2025-12-13 20:15:38 - ras_commander.check.report - INFO - Generated HTML report: c:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\ras_checker\validation_report.html


HTML report generated: c:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\ras_checker\validation_report.html


### HTML Report with Metadata

Add project metadata for more informative reports:

In [21]:
# Create report with metadata
metadata = ReportMetadata(
    project_name=ras.project_name,
    project_path=project_path,
    plan_name="Steady Flow Run"
)

# Generate report with metadata
report_path_meta = project_path / "ras_checker" / "validation_report_with_metadata.html"
output_path = results.to_html(report_path_meta, metadata=metadata)
print(f"Report with metadata generated: {output_path}")

2025-12-13 20:21:38 - ras_commander.check.report - INFO - Generated HTML report: c:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\ras_checker\validation_report_with_metadata.html


Report with metadata generated: c:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\ras_checker\validation_report_with_metadata.html


### Export to CSV

Export messages to CSV for further analysis in Excel or other tools:

In [22]:
# Export to CSV
csv_path = project_path / "ras_checker" / "validation_messages.csv"

df = results.to_dataframe()
if not df.empty:
    df.to_csv(csv_path, index=False)
    print(f"CSV exported: {csv_path}")
    print(f"Exported {len(df)} messages")
else:
    print("No messages to export")

CSV exported: c:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\ras_checker\validation_messages.csv
Exported 4145 messages


---

## Customizing Validation Thresholds

RasCheck uses configurable thresholds for all checks. You can customize these for project-specific requirements.

In [25]:
# View default thresholds
defaults = get_default_thresholds()

print("Default Manning's n Thresholds:")
print(f"  Overbank: {defaults.mannings_n.overbank_min} - {defaults.mannings_n.overbank_max}")
print(f"  Channel:  {defaults.mannings_n.channel_min} - {defaults.mannings_n.channel_max}")

print("\nDefault Transition Coefficients:")
print(f"  Structures: Contraction={defaults.transitions.structure_contraction_max}, Expansion={defaults.transitions.structure_expansion_max}")
print(f"  Regular XS: Contraction={defaults.transitions.regular_contraction_max}, Expansion={defaults.transitions.regular_expansion_max}")

Default Manning's n Thresholds:
  Overbank: 0.03 - 0.2
  Channel:  0.025 - 0.1

Default Transition Coefficients:
  Structures: Contraction=0.3, Expansion=0.5
  Regular XS: Contraction=0.1, Expansion=0.3


In [26]:
# Create custom thresholds
custom_thresholds = create_custom_thresholds({
    # Stricter Manning's n limits
    'mannings_n.overbank_min': 0.035,
    'mannings_n.overbank_max': 0.150,
    'mannings_n.channel_min': 0.028,
    'mannings_n.channel_max': 0.080,
    
    # Custom reach length limits
    'reach_length.max_length_ft': 2000.0,
    'reach_length.length_ratio_max': 1.5,
})

print("Custom thresholds created")
print(f"  Custom overbank min: {custom_thresholds.mannings_n.overbank_min}")

Custom thresholds created
  Custom overbank min: 0.035


In [27]:
# Run checks with custom thresholds
custom_results = RasCheck.run_all(plan_number, thresholds=custom_thresholds)

print(f"\nResults with custom thresholds:")
print(f"  Total Messages: {len(custom_results.messages)}")
print(f"  Errors: {custom_results.get_error_count()}")
print(f"  Warnings: {custom_results.get_warning_count()}")

print(f"\nComparison with default thresholds:")
print(f"  Default: {len(results.messages)} messages")
print(f"  Custom:  {len(custom_results.messages)} messages")

2025-12-13 21:59:11 - ras_commander.hdf.HdfXsec - ERROR - Error processing cross-section data: [Errno 2] Unable to synchronously open file (unable to open file: name = 'C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.hdf', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
2025-12-13 21:59:11 - ras_commander.hdf.HdfXsec - ERROR - Error processing cross-section data: [Errno 2] Unable to synchronously open file (unable to open file: name = 'C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.hdf', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
2025-12-13 21:59:11 - ras_commander.hdf.HdfStruc - ERROR - HDF file not found: C:\GH\ras-commander\examples\example_projects_28_quality_assurance_rascheck\Balde Eagle Creek\BaldEagle.hdf
2025-12-13 21:59:11 - ras_commander.check.RasCheck - WARNING - Could not read structures: HDF f


Results with custom thresholds:
  Total Messages: 4145
  Errors: 1
  Warnings: 3671

Comparison with default thresholds:
  Default: 4145 messages
  Custom:  4145 messages


---

## State-Specific Floodway Surcharge Limits

Different states have different maximum surcharge limits for floodway analysis:

In [28]:
# View state-specific surcharge limits
print("State-Specific Surcharge Limits:")
print("="*40)

states = ['TX', 'IL', 'WI', 'MN', 'NJ', 'MI', 'IN', 'OH']
for state in states:
    limit = get_state_surcharge_limit(state)
    print(f"  {state}: {limit:.2f} ft")

State-Specific Surcharge Limits:
  TX: 1.00 ft
  IL: 0.10 ft
  WI: 0.01 ft
  MN: 0.50 ft
  NJ: 0.20 ft
  MI: 1.00 ft
  IN: 1.00 ft
  OH: 1.00 ft


In [29]:
# Example: Using Illinois standards (stricter 0.1 ft surcharge)
il_limit = get_state_surcharge_limit('IL')
print(f"Illinois surcharge limit: {il_limit} ft")

# You would use this when running floodway checks:
# results = RasCheck.run_all(
#     plan_number,
#     floodway_profile='Floodway',
#     surcharge=il_limit
# )

Illinois surcharge limit: 0.1 ft


---

## Understanding Message IDs

RasCheck uses a standardized message ID format:

```
{CHECK_TYPE}_{CATEGORY}_{NUMBER}{SUFFIX}
```

Examples:
- `NT_RC_01L` - NT Check, Roughness Coefficient, Message 01, Left side
- `XS_DT_02R` - XS Check, Distance/Travel, Message 02, Right side
- `BR_TF_04` - Bridge, Type Flow, Message 04

In [30]:
# Analyze message ID distribution
if not df.empty:
    print("Message ID Distribution:")
    print("="*50)
    msg_counts = df['message_id'].value_counts()
    print(msg_counts.head(15))

Message ID Distribution:
message_id
MP_WS_01     1246
MP_Q_01      1246
PF_VEL_01     895
MP_TW_01      473
PF_TW_01      284
SYS_003         1
Name: count, dtype: int64


---

## Summary

This notebook demonstrated how to use RasCheck for HEC-RAS model validation:

1. **Initialization**: Extract and initialize a HEC-RAS project
2. **Running Checks**: Use `RasCheck.run_all()` or individual check methods
3. **Analyzing Results**: Filter by severity, check type, or station
4. **Generating Reports**: Create HTML reports and CSV exports
5. **Custom Thresholds**: Adjust validation criteria for project requirements
6. **State-Specific Standards**: Use appropriate surcharge limits

### Key Classes and Methods

| Class/Method | Description |
|--------------|-------------|
| `RasCheck.run_all()` | Run all validation checks |
| `RasCheck.check_nt()` | Manning's n and transition checks |
| `RasCheck.check_xs()` | Cross section validation |
| `RasCheck.check_structures()` | Bridge/culvert/weir checks |
| `RasCheck.check_floodways()` | Floodway surcharge validation |
| `RasCheck.check_profiles()` | Multiple profile consistency |
| `CheckResults.to_html()` | Generate HTML report |
| `CheckResults.to_dataframe()` | Convert to pandas DataFrame |
| `create_custom_thresholds()` | Create custom validation criteria |
| `get_state_surcharge_limit()` | Get state-specific floodway limits |

### For More Information

See the RasCheck documentation pages for detailed message references:
- NT Check (Manning's n)
- XS Check (Cross Sections)
- Structure Check
- Floodway Check
- Profiles Check